In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch

torch.cuda.device_count()

1

In [3]:
import torch

torch.cuda.get_device_capability()

(7, 5)

In [4]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

In [7]:
from lightning import Trainer
from src.model import HousePricingModel
from src.data import HousePricingDataModule

data_module = HousePricingDataModule()
data_module.prepare_data()

model = HousePricingModel()

trainer = Trainer(accelerator="gpu")

trainer.fit(model, train_dataloaders=data_module.train_dataloader(), val_dataloaders=data_module.val_dataloader())

ValueError: Found unknown categories ['Membran'] in column 13 during transform

In [ ]:
data_module.in_features()

37